## Datasets
* AS (Traditional Chinese) http://www.khngai.com/chinese/charmap/tblbig.php?page=0
* CITYU (Traditional Chinese)
* MSR (Simplified Chinese)
* PKU (Simplified Chinese)


- Each dataset has one train and one dev set
- You can find the train datasets in the training folder
- Your dev set is in the gold folder
- For each dataset use the file with the .utf8 extension
 

## TO DO
1. read original file in a train/dev/test fodler
2. produce an input file with NO spaces
3. and a label file in BIES format

In [155]:
datasets = {"training":'../icwb2-data/training',
             "dev":'../icwb2-data/gold',
             "testing":'../icwb2-data/testing'}

In [163]:
class Preprocess(object):
    def __init__(self, dataset_path):
        self.dataset_path = dataset_path
        
    def deleteFiles(self, extension = '.txt'):
        '''
        Deletes all the .txt files in a given directory
        Only done to get rid of duplicate .txt files as I am using .utf8 files for training
        '''
        print("Deleteing {} files from {}".format(extension, self.dataset_path))
        for path, subdir, files in os.walk(self.dataset_path):
            for name in files: 
                if name.endswith(extension):
                    os.system('rm '+ os.path.join(self.dataset_path,name))
                    print('Succesfully removed: {}'.format(name))
            print("Done\n")

    def hanzi_convert(self, input_file):
        '''transoforms a file form Traditional to Simplified Chinese'''
        
        _, _, name_, format_ = input_file.split(".")
        file_name_input = name_.split('/')[-1]
        name_ += '_simplified.' + format_
        file_name_output = name_.split('/')[-1]

        bash_ = 'hanzi-convert -s ' + input_file + ' > ' + name_
        os.system(bash_)
        print("Created {}".format(file_name_output))
        
    def traditionalToSimplified(self):
        count=0
        len_ = len(os.listdir(self.dataset_path))
        print("start for path: {}\n".format(self.dataset_path))
        for path, subdir, files in os.walk(self.dataset_path):
            for name in files: 
                x = os.path.join(self.dataset_path, name)
                self.hanzi_convert(x)
                count+=1
                print(">>>{}/{} complete============\n".format(count, len_))
        print("end\n")
#         return None
    
    def action(self, action_type):
        count=0
        len_ = len(os.listdir(self.dataset_path))
        print("start for path: {}\n".format(self.dataset_path))
        for path, subdir, files in os.walk(self.dataset_path):
            for name in files: 
                x = os.path.join(self.dataset_path, name)
                print("File: {}".format(x))
                if action_type == 'translate': 
                    self.hanzi_convert(x)
                elif action_type == 'InputFile':
                    self.InputFile(x)
                elif action_type == 'LabelFile':
                    self.FileToBIES(x)   
                count+=1
                print(">>>{}/{} complete============\n".format(count, len_))
        print("end\n")
        return None
    
    
    
    def InputFile(self, input_file):
        '''creates a file to feed to the TF model by getting rid of spaces'''
        pass
    
    
    
    
    @staticmethod
    def encoder(line, type_of_file, verbose = False):
        '''encodes a single line'''

        new_line = ''

        #uses the +u3000 seperator if the file is 'AS'
        words = line.rstrip().split("\u3000") if type_of_file == 'as' else line.rstrip().split(" ")
        #print("N of segments:",len(words))

        def bies_format(word):
            '''changes word to BIES format'''
            return 'B'+'I'*int(len(word)-2)+'E' if len(word)>1 else 'S'

        for word in words:
            if verbose: print("this is a word: {} of length {}".format(word, len(word)))
            new_line+=bies_format(word)

        return new_line

    def FileToBIES(self, file_path):
        '''gives a file in utf converts to a txt in with all lines in the BIES format'''


        #print("HERE:",file_path)
        name_ = file_path.split("/")[-1].split(".")[0] + '_labels'
        #print("NAME",name_, file_path)
        type_of_file = name_.split("/")[-1].split('_')[0] #to see if its an 'AS' file

        with open(file_path, 'r', encoding ='utf8') as f1, open(name_+".txt", "w") as f2:
            c=0
            for line in f1:
                c+=1
                #print("input line:\t",line.rstrip())
                out = encoder(line, type_of_file)
                #print("output line:\t{}".format(out))
                f2.writelines(out+"\n")
        return None
    

In [164]:
import os
P = Preprocess(datasets['training'])
P.deleteFiles('.txt')
P.deletefiles('.b5')
#P.traditionalToSimplified()
P.action('LabelFile')

Deleteing .txt files from ../icwb2-data/training
Done



AttributeError: 'Preprocess' object has no attribute 'deletefiles'

In [3]:
x = '植物　研究所　所长　周昌弘　先生　当选　第三世界　科学院　（　Ｔｈｅ　Ｔｈｉｒｄ　Ｗｏｒｌｄ　Ａｃａｄｅｍｙ　ｏｆ　Ｓｃｉｅｎｃｅｓ　，'

In [16]:
x.rstrip().split("\u3000")

['植物',
 '研究所',
 '所长',
 '周昌弘',
 '先生',
 '当选',
 '第三世界',
 '科学院',
 '（',
 'Ｔｈｅ',
 'Ｔｈｉｒｄ',
 'Ｗｏｒｌｄ',
 'Ａｃａｄｅｍｙ',
 'ｏｆ',
 'Ｓｃｉｅｎｃｅｓ',
 '，']

In [14]:
eng = x.rstrip().split("\u3000")[10]
eng

'Ｔｈｉｒｄ'

In [18]:
eng[0],eng[1]

('Ｔ', 'ｈ')